# V7: industry1 (layer1 = vertical,layer2 =  horizontal)

In [9]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
filepath_out='../output/industry1.router'
netlist_file_path = '../benchmark/industry1.nl'
gridfile_path  = '../benchmark/industry1.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_grid_original,layer2_grid_original = parse_gridfile(gridfile_path)

bend_penalty: 10 
 via_penalty: 20
size of layer1: (739, 910)
size of layer2: (739, 910)


In [10]:
# plot problem 根据grid文件可视化障碍物和每个格子的cost，同时标出source和target，可以显示两层
plot_problem("../output/industry1_problem.jpg",columns,rows,layer1_grid_original,layer2_grid_original,nets)

In [ ]:
def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def mark_path_on_grid(layer1_grid, path):
    if path:               
        for cell in path:
            x, y, _ = cell
            layer1_grid[x][y] = -1

def get_cell_cost(layer_grid, cell,path_tmp,bend_penalty,via_penalty):  
    x, y, layer = cell
    cell_cost = 1  # 默认的单元代价
    if layer_grid[x][y] == -1:
        cell_cost = float('inf')  # -1表示无法通过的细胞
    elif layer_grid[x][y] != 1:
        cell_cost = layer_grid[x][y]  # 非单元代价
    if len(path_tmp) >= 2:
        prev_cell = path_tmp[-2]
        prev_x, prev_y, prev_layer = prev_cell
        if layer != prev_layer:
            cell_cost += via_penalty
        elif (prev_x != x and prev_y != y):  
            cell_cost += bend_penalty
    return cell_cost

def expand_source_to_target(rows, columns, layer1_grid,layer2_grid, source, target,bend_penalty,via_penalty):
    wavefront = {}
    visited = set()
    parents = {}
    costs = {}  # Store the cumulative costs for each cell
    
    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    wavefront[source_tuple] = 0
    costs[source_tuple] = 1  # Initial cost for the source cell is 1

    while wavefront:
        # get lowest cost cell on a wavefront structure
        current_cell = sorted(wavefront.items(),key=lambda s:int(s[1]))[0][0]

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path,costs[current_cell]

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])
            if neighbor_tuple not in visited:
                path_tmp= reconstruct_path(source_tuple, current_cell, parents)
                # Calculate the cost to reach the neighbor cell
                if(neighbor_tuple[2]== 1):
                  cost = costs[current_cell] + get_cell_cost(layer1_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                if(neighbor_tuple[2]== 2):
                  cost = costs[current_cell] + get_cell_cost(layer2_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                # ignore blocks
                if cost!= np.inf:
                    if neighbor_tuple not in wavefront.keys() or costs[neighbor_tuple] > cost:
                        costs[neighbor_tuple] = cost
                        parents[neighbor_tuple] = current_cell
                    if neighbor_tuple not in wavefront.keys():
                        # add cell N to waveform, indexed by pathcost
                        wavefront[neighbor_tuple]=cost         

        visited.add(current_cell)    
        del wavefront[current_cell]                  
    return None,None

def get_neighbors(rows, columns, cell):  
    x, y, layer = cell
    neighbors = []
    if layer == 1: 
        if x < columns - 1:
            neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
        if x > 0:
            neighbors.append({'x': x - 1, 'y': y, 'layer': layer})  
        neighbors.append({'x': x, 'y': y, 'layer': 3-layer})    
    elif layer == 2: 
        if y < rows - 1:
            neighbors.append({'x': x, 'y': y + 1, 'layer': layer})
        if y > 0:
            neighbors.append({'x': x, 'y': y - 1, 'layer': layer})  
        neighbors.append({'x': x, 'y': y, 'layer': 3-layer})                                       
    return neighbors

def true_two_layer_router(rows, columns, layer1_grid,layer2_grid, nets,bend_penalty,via_penalty):
    nets.sort(key=lambda s: ((s["pin1"]['x']-s["pin2"]['x'])**2+(s["pin1"]['y']-s["pin2"]['y'])**2))
    routing_table = {}
    costs_table = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        # 防止布线在后续的pin上，先将所有的pin标记为-1；
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = -1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = -1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = -1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = -1
           
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = 1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = 1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = 1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = 1
        print('Routing net:',net_id)
        path,costs = expand_source_to_target(rows, columns, layer1_grid,layer2_grid, pin1, pin2,bend_penalty,via_penalty) 
        if path is not None:
            mark_path_on_grid(layer1_grid,path)
            routing_table[net_id] = path
            costs_table[net_id] = costs
    routing_table = dict(sorted(routing_table.items(),key=lambda x:x[0]))
    costs_table = dict(sorted(costs_table.items(),key=lambda x:x[0]))
    return routing_table,costs_table

In [ ]:
layer1_grid = layer1_grid_original.copy()
layer2_grid = layer2_grid_original.copy()
routing_table,costs_table=true_two_layer_router(rows, columns, layer1_grid.T,layer2_grid.T, nets,bend_penalty,via_penalty)
plot_path('../output/industry1.jpg',columns=columns,rows=rows,grid1=layer1_grid_original,grid2=layer2_grid_original,path_dict=routing_table) 
generate_output_file(filepath_out,net_num,routing_table)

Routing net: 440
Routing net: 662
Routing net: 713
Routing net: 130
Routing net: 247
Routing net: 41
Routing net: 593
Routing net: 285
Routing net: 910
Routing net: 981
Routing net: 731
Routing net: 967
Routing net: 143
Routing net: 881
Routing net: 919
Routing net: 963
Routing net: 717
Routing net: 691
Routing net: 804
Routing net: 171
Routing net: 576
Routing net: 641
Routing net: 606
Routing net: 407
Routing net: 760
Routing net: 495
Routing net: 520
Routing net: 521
Routing net: 522
Routing net: 277
Routing net: 567
Routing net: 137
Routing net: 480
Routing net: 965
Routing net: 884
Routing net: 344
Routing net: 631
Routing net: 933
Routing net: 297
Routing net: 420
Routing net: 883
Routing net: 614
Routing net: 421
Routing net: 16
Routing net: 900
Routing net: 422
Routing net: 592
Routing net: 671
Routing net: 109
Routing net: 585
Routing net: 246
Routing net: 213
Routing net: 724
Routing net: 801
Routing net: 931
Routing net: 819
Routing net: 139
Routing net: 992
Routing net: 425

KeyboardInterrupt: 

In [ ]:
costs_table

{95: 23,
 109: 23,
 119: 23,
 64: 45,
 74: 45,
 105: 45,
 14: 89,
 20: 89,
 22: 89,
 40: 89,
 84: 111,
 12: 155,
 38: 155,
 80: 166,
 42: 218,
 51: 188,
 79: 188,
 103: 188,
 9: 199,
 53: 199,
 91: 199,
 33: 210,
 67: 210,
 117: 210,
 13: 221,
 94: 221,
 101: 221,
 116: 221,
 102: 232,
 4: 254,
 123: 254,
 11: 243,
 37: 241,
 78: 265,
 111: 265,
 113: 287,
 112: 318,
 26: 287,
 107: 287,
 108: 340,
 118: 243,
 81: 309,
 31: 331,
 76: 331,
 100: 342,
 110: 394,
 106: 364,
 115: 317,
 49: 298,
 114: 328,
 44: 419,
 92: 329,
 27: 430,
 10: 331,
 89: 441,
 54: 342,
 63: 342,
 120: 342,
 45: 353,
 65: 496,
 1: 364,
 2: 364,
 24: 527,
 50: 386,
 6: 560,
 46: 472,
 52: 430,
 73: 430,
 34: 493,
 96: 526,
 72: 449,
 18: 573,
 77: 573,
 87: 518,
 88: 571,
 32: 604,
 68: 614,
 75: 582,
 25: 441,
 48: 474,
 121: 452,
 122: 452,
 61: 658,
 29: 617,
 71: 658,
 69: 526,
 98: 526,
 104: 516,
 47: 538,
 15: 714,
 86: 812,
 8: 782,
 85: 615,
 36: 626,
 7: 639,
 99: 727,
 82: 813,
 59: 648,
 39: 659,
 57

In [ ]:
cnt = 0
for item in routing_table.keys():
    if routing_table[item]:
        cnt += 1
cnt

125